In [1]:
import pandas as pd
import numpy as np
import tempfile
import os.path
import calendar
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import geocoder

In [2]:
# ouverture du DataFrame sans valeurs manquantes non reconnaissables par Pandas
missing_values = ["n/a", "na", "--", "-"]
dataset = pd.read_csv(r"D:\_IMSD\Data viz'\dataset.csv", encoding="ISO-8859-1", na_values = missing_values)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# copy of DataFrame => cleaned one
df = dataset.copy()
df.head()
df.index

RangeIndex(start=0, stop=181691, step=1)

In [4]:
pd.set_option('display.max_columns', None)
display(df)

eventid  iyear  imonth  iday           approxdate  extended  \
0       197000000001   1970       7     2                  NaN         0   
1       197000000002   1970       0     0                  NaN         0   
2       197001000001   1970       1     0                  NaN         0   
3       197001000002   1970       1     0                  NaN         0   
4       197001000003   1970       1     0                  NaN         0   
5       197001010002   1970       1     1                  NaN         0   
6       197001020001   1970       1     2                  NaN         0   
7       197001020002   1970       1     2                  NaN         0   
8       197001020003   1970       1     2                  NaN         0   
9       197001030001   1970       1     3                  NaN         0   
10      197001050001   1970       1     1                  NaN         0   
11      197001060001   1970       1     6                  NaN         0   
12      197001080001   1970       1     8                  NaN         0   
13      197001090001   1970       1     9                  NaN         0   
14      197001090002   1970       1     9                  NaN         0   
15      197001100001   1970       1    10                  NaN         0   
16      197001110001   1970       1    11                  NaN         0   
17      197001120001   1970       1    12                  NaN         0   
18      197001120002   1970       1    12                  NaN         0   
19      197001130001   1970       1    13                  NaN         0   
20      197001140001   1970       1    14                  NaN         0   
21      197001150001   1970       1    15                  NaN         0   
22      197001190002   1970       1    19                  NaN         0   
23      197001190003   1970       1    19                  NaN         0   
24      197001190004   1970       1    19  January 19-20, 1970         0   
25      197001200001   1970       1    20                  NaN         0   
26      197001210001   1970       1    21                  NaN         0   
27      197001220001   1970       1    22                  NaN         0   
28      197001220002   1970       1    22                  NaN         0   
29      197001250001   1970       1    25                  NaN         0   
...              ...    ...     ...   ...                  ...       ...   
181661  201712300019   2017      12    28                  NaN         0   
181662  201712300020   2017      12    30                  NaN         0   
181663  201712300021   2017      12    30                  NaN         0   
181664  201712300022   2017      12    30                  NaN         0   
181665  201712300023   2017      12    30                  NaN         0   
181666  201712300025   2017      12    30                  NaN         0   
181667  201712300026   2017      12    30                  NaN         0   
181668  201712310001   2017      12    31  2017-12-31 00:00:00         0   
181669  201712310002   2017      12    30                  NaN         0   
181670  201712310003   2017      12    31  2017-12-31 00:00:00         0   
181671  201712310004   2017      12    31                  NaN         0   
181672  201712310005   2017      12    31                  NaN         0   
181673  201712310006   2017      12    31                  NaN         0   
181674  201712310007   2017      12    31                  NaN         0   
181675  201712310008   2017      12    31                  NaN         0   
181676  201712310009   2017      12    31                  NaN         0   
181677  201712310010   2017      12    31                  NaN         0   
181678  201712310011   2017      12    30                  NaN         0   
181679  201712310012   2017      12    31                  NaN         0   
181680  201712310013   2017      12    31                  NaN         0   
181681  201712310016   2017      12    31                  Na

In [8]:
'''for i in df.columns:
    print(i)'''

'for i in df.columns:\n    print(i)'

In [17]:
filtered_df = df[df['addnotes'].notnull()]
filtered_df

eventid  iyear imonth  iday           approxdate  extended  \
5       197001010002   1970    Jan     1                  NaN         0   
7       197001020002   1970    Jan     2                  NaN         0   
8       197001020003   1970    Jan     2                  NaN         0   
9       197001030001   1970    Jan     3                  NaN         0   
14      197001090002   1970    Jan     9                  NaN         0   
17      197001120001   1970    Jan    12                  NaN         0   
19      197001130001   1970    Jan    13                  NaN         0   
22      197001190002   1970    Jan    19                  NaN         0   
23      197001190003   1970    Jan    19                  NaN         0   
24      197001190004   1970    Jan    19  January 19-20, 1970         0   
28      197001220002   1970    Jan    22                  NaN         0   
29      197001250001   1970    Jan    25                  NaN         0   
30      197001250002   1970    Jan    25                  NaN         0   
31      197001260001   1970    Jan    26                  NaN         0   
32      197001260003   1970    Jan    26                  NaN         0   
33      197001270002   1970    Jan    27                  NaN         0   
36      197001300001   1970    Jan    30                  NaN         0   
37      197001300002   1970    Jan    30                  NaN         0   
41      197002010001   1970    Feb     1                  NaN         0   
42      197002010002   1970    Feb     1                  NaN         0   
44      197002030001   1970    Feb     3                  NaN         0   
45      197002030002   1970    Feb     3                  NaN         0   
46      197002040001   1970    Feb     4                  NaN         0   
47      197002060001   1970    Feb     6                  NaN         0   
48      197002060002   1970    Feb     6                  NaN         0   
49      197002060003   1970    Feb     6                  NaN         0   
50      197002060004   1970    Feb     6                  NaN         0   
51      197002060005   1970    Feb     6                  NaN         0   
52      197002060006   1970    Feb     6                  NaN         0   
54      197002070003   1970    Feb     7                  NaN         0   
...              ...    ...    ...   ...                  ...       ...   
181591  201712280004   2017    Dec    28                  NaN         0   
181592  201712280005   2017    Dec    28                  NaN         0   
181602  201712280019   2017    Dec    28                  NaN         1   
181605  201712280022   2017    Dec    28                  NaN         0   
181608  201712280026   2017    Dec    28                  NaN         0   
181610  201712280029   2017    Dec    28                  NaN         0   
181611  201712280030   2017    Dec    28                  NaN         0   
181614  201712280034   2017    Dec    28                  NaN         0   
181623  201712290001   2017    Dec    29                  NaN         0   
181624  201712290002   2017    Dec    29                  NaN         0   
181629  201712290007   2017    Dec    28  2017-12-28 00:00:00         0   
181631  201712290009   2017    Dec    30                  NaN         0   
181637  201712290022   2017    Dec    29                  NaN         0   
181638  201712290023   2017    Dec    29                  NaN         0   
181639  201712290024   2017    Dec    29                  NaN         0   
181641  201712290026   2017    Dec    24                  NaN         0   
181649  201712300006   2017    Dec    30                  NaN         0   
181652  201712300010   2017    Dec    30                  NaN         0   
181653  201712300011   2017    Dec    31                  NaN         0   
181657  201712300015   2017    Dec    30                  NaN         0   
181658  201712300016   2017    Dec    30                  NaN         0   
181659  201712300017   2017    Dec    

# Suppression des attaques n'ayant pas de date complète

#### Suppression des lignes n'ayant pas de mois ou de jours (0.49%)

In [39]:
#df[df['imonth'] == 0].count()
#df[df['iday'] == 0].count()
print((891/181691)*100)

0.4903930299244321


In [5]:
df.drop(df[(df.imonth == 0) | (df.iday ==0)].index, inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180800 entries, 0 to 181690
Columns: 135 entries, eventid to related
dtypes: float64(55), int64(22), object(58)
memory usage: 187.6+ MB


# Remplacer les numéros de mois par leur abbréviation

In [7]:
df['imonth'] = df['imonth'].apply(lambda x: calendar.month_abbr[x])

In [8]:
df.head()

eventid  iyear imonth  iday approxdate  extended resolution  country  \
0  197000000001   1970    Jul     2        NaN         0        NaN       58   
5  197001010002   1970    Jan     1        NaN         0        NaN      217   
6  197001020001   1970    Jan     2        NaN         0        NaN      218   
7  197001020002   1970    Jan     2        NaN         0        NaN      217   
8  197001020003   1970    Jan     2        NaN         0        NaN      217   

          country_txt  region                   region_txt   provstate  \
0  Dominican Republic       2  Central America & Caribbean         NaN   
5       United States       1                North America    Illinois   
6             Uruguay       3                South America  Montevideo   
7       United States       1                North America  California   
8       United States       1                North America   Wisconsin   

            city   latitude   longitude  specificity  vicinity  \
0  Santo Domingo  18.456792  -69.951164          1.0         0   
5          Cairo  37.005105  -89.176269          1.0         0   
6     Montevideo -34.891151  -56.187214          1.0         0   
7        Oakland  37.791927 -122.225906          1.0         0   
8        Madison  43.076592  -89.412488          1.0         0   

          location                                            summary  crit1  \
0              NaN                                                NaN      1   
5              NaN  1/1/1970: Unknown African American assailants ...      1   
6              NaN                                                NaN      1   
7  Edes Substation  1/2/1970: Unknown perpetrators detonated explo...      1   
8              NaN  1/2/1970: Karl Armstrong, a member of the New ...      1   

   crit2  crit3  doubtterr  alternative   alternative_txt  multiple  success  \
0      1      1        0.0          NaN               NaN       0.0        1   
5      1      1        0.0          NaN               NaN       0.0        1   
6      1      1        0.0          NaN               NaN       0.0        0   
7      1      1        1.0          2.0  Other Crime Type       0.0        1   
8      1      1        0.0          NaN               NaN       0.0        1   

   suicide  attacktype1                 attacktype1_txt  attacktype2  \
0        0            1                   Assassination          NaN   
5        0            2                   Armed Assault          NaN   
6        0            1                   Assassination          NaN   
7        0            3               Bombing/Explosion          NaN   
8        0            7  Facility/Infrastructure Attack          NaN   

  attacktype2_txt  attacktype3 attacktype3_txt  targtype1  \
0             NaN          NaN             NaN         14   
5             NaN          NaN             NaN          3   
6             NaN          NaN             NaN          3   
7             NaN          NaN             NaN         21   
8             NaN          NaN             NaN          4   

                 targtype1_txt  targsubtype1  \
0  Private Citizens & Property          68.0   
5                       Police          22.0   
6                       Police          25.0   
7                    Utilities         107.0   
8                     Military          28.0   

                                  targsubtype1_txt  \
0                                   Named Civilian   
5  Police Building (headquarters, station, school)   
6                  Police Security Forces/Officers   
7                                      Electricity   
8              Military Recruiting Station/Academy   

                            corp1  \
0                             NaN   
5         Cairo Police Department   
6                Uruguayan Police   
7  Pacific Gas & Electric Company   
8                        R.O.T.C.   

                                             target1  natlty1  \
0                    

# Conserver uniquement data > 1997

In [9]:
higher_1997 = df.copy()

In [14]:
data = higher_1997[higher_1997['iyear'] >= 1997]
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117110 entries, 64310 to 181690
Columns: 135 entries, eventid to related
dtypes: float64(55), int64(21), object(59)
memory usage: 121.5+ MB


# Suppression des colonnes non utilisées pour l'analyse

In [15]:
data.drop([
    'country', 'region','specificity','vicinity','doubtterr','alternative','alternative_txt', 'attacktype1', 'attacktype2',
    'attacktype3', 'targtype1', 'targtype2', 'targtype3', 'targsubtype1', 'targsubtype2', 'targsubtype3', 'natlty1',
    'natlty2', 'natlty3', 'claimmode', 'claimmode2', 'claimmode3', 'weaptype1', 'weaptype1', 'weaptype2', 'weaptype3',
    'weaptype4', 'weapsubtype1', 'weapsubtype2', 'weapsubtype3', 'weapsubtype4', 'nkillus', 'nwoundus', 'nhostkidus',
    'ransomamtus', 'hostkidoutcome', 'INT_LOG', 'INT_IDEO', 'INT_MISC', 'INT_ANY', 'related'    
], axis=1, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


# Renomer les colonnes

In [16]:
columns = [i for i in data.columns]
print(columns)

['eventid', 'iyear', 'imonth', 'iday', 'approxdate', 'extended', 'resolution', 'country_txt', 'region_txt', 'provstate', 'city', 'latitude', 'longitude', 'location', 'summary', 'crit1', 'crit2', 'crit3', 'multiple', 'success', 'suicide', 'attacktype1_txt', 'attacktype2_txt', 'attacktype3_txt', 'targtype1_txt', 'targsubtype1_txt', 'corp1', 'target1', 'natlty1_txt', 'targtype2_txt', 'targsubtype2_txt', 'corp2', 'target2', 'natlty2_txt', 'targtype3_txt', 'targsubtype3_txt', 'corp3', 'target3', 'natlty3_txt', 'gname', 'gsubname', 'gname2', 'gsubname2', 'gname3', 'gsubname3', 'motive', 'guncertain1', 'guncertain2', 'guncertain3', 'individual', 'nperps', 'nperpcap', 'claimed', 'claimmode_txt', 'claim2', 'claimmode2_txt', 'claim3', 'claimmode3_txt', 'compclaim', 'weaptype1_txt', 'weapsubtype1_txt', 'weaptype2_txt', 'weapsubtype2_txt', 'weaptype3_txt', 'weapsubtype3_txt', 'weaptype4_txt', 'weapsubtype4_txt', 'weapdetail', 'nkill', 'nkillter', 'nwound', 'nwoundte', 'property', 'propextent', 'pr

# Création du dataset pour le random button

In [46]:
random_button_2 = data.dropna()

In [48]:
random_button.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102371 entries, 64315 to 181689
Data columns (total 95 columns):
eventid               102371 non-null int64
iyear                 102371 non-null int64
imonth                102371 non-null object
iday                  102371 non-null int64
approxdate            6814 non-null object
extended              102371 non-null int64
resolution            811 non-null object
country_txt           102371 non-null object
region_txt            102371 non-null object
provstate             102362 non-null object
city                  102371 non-null object
latitude              101618 non-null float64
longitude             101618 non-null float64
location              49437 non-null object
summary               102371 non-null object
crit1                 102371 non-null int64
crit2                 102371 non-null int64
crit3                 102371 non-null int64
multiple              102370 non-null float64
success               102371 non-null in

# Standardisation des pays

In [23]:
'''test = data.copy()

czechoslovakia = test[test['country_txt'] == 'Czechoslovakia']
czechoslovakia'''

"test = data.copy()\n\nczechoslovakia = test[test['country_txt'] == 'Czechoslovakia']\nczechoslovakia"

In [11]:
czech_republic = {'Czech Republic': ['Prague', 'Usti Nad Labem', 'Halickuv Brod', 'Hradec Kralove']} # replace from Czechoslovakia
slovakia = {'Slovak Republic': ['Bratislava']} # Czechoslovakia
serbia = {'Serbia': ['Belgrade', 'Bujanovac', 'Vrbas', 'Lucani', 'Bujanovac', 'Dobrosin']} # replace from Serbia-Montenegro & Yugoslavia
montenegro = {'Montenegro': ['Podgorica', 'Berane', 'Podgorica', 'Kolasin']} # replace from Serbia-Montenegro & Yugoslavia

In [19]:
yugoslavia = test[test['country_txt'] == 'Yugoslavia']
yugoslavia.s

eventid  iyear imonth  iday approxdate  extended resolution  \
1970   197308040001   1973    Aug     4        NaN         0        NaN   
2457   197406240004   1974    Jun    24        NaN         0        NaN   
4922   197706200005   1977    Jun    20        NaN         0        NaN   
10781  198004220008   1980    Apr    22        NaN         0        NaN   
13700  198106100004   1981    Jun    10        NaN         0        NaN   
14425  198109260001   1981    Sep    26        NaN         0        NaN   
18257  198303090004   1983    Mar     9        NaN         0        NaN   
22249  198406090002   1984    Jun     9        NaN         0        NaN   
37774  198904020001   1989    Apr     2        NaN         0        NaN   
37775  198904020002   1989    Apr     2        NaN         0        NaN   
38123  198905010010   1989    May     1        NaN         0        NaN   
41538  199002220010   1990    Feb    22        NaN         0        NaN   
41755  199003180005   1990    Mar    18        NaN         0        NaN   
44293  199010160019   1990    Oct    16        NaN         0        NaN   
44346  199010200006   1990    Oct    20        NaN         0        NaN   
44468  199010310006   1990    Oct    31        NaN         0        NaN   
45746  199103100003   1991    Mar    10        NaN         0        NaN   
46291  199104300005   1991    Apr    30        NaN         0        NaN   
46314  199105020007   1991    May     2        NaN         0        NaN   
46329  199105040009   1991    May     4        NaN         0        NaN   
46730  199106080005   1991    Jun     8        NaN         0        NaN   
46736  199106080011   1991    Jun     8        NaN         0        NaN   
46763  199106100007   1991    Jun    10        NaN         0        NaN   
46916  199106200007   1991    Jun    20        NaN         0        NaN   
47282  199107170012   1991    Jul    17        NaN         0        NaN   
47492  199107310001   1991    Jul    31        NaN         0        NaN   
49636  199112300015   1991    Dec    30        NaN         0        NaN   
50298  199202240010   1992    Feb    24        NaN         0        NaN   
50412  199203060005   1992    Mar     6        NaN         0        NaN   
50459  199203100012   1992    Mar    10        NaN         0        NaN   
...             ...    ...    ...   ...        ...       ...        ...   
70372  200005130002   2000    May    13        NaN         0        NaN   
70456  200005310002   2000    May    31        NaN         0        NaN   
70520  200006080002   2000    Jun     8        NaN         0        NaN   
70565  200006150003   2000    Jun    15        NaN         0        NaN   
70589  200006210004   2000    Jun    21        NaN         0        NaN   
71170  200010110002   2000    Oct    10        NaN         0        NaN   
71208  200010170011   2000    Oct    17        NaN         0        NaN   
71656  200101020003   2001    Jan     2        NaN         0        NaN   
71777  200101240002   2001    Jan    24        NaN         0        NaN   
71778  200101240003   2001    Jan    24        NaN         0        NaN   
71787  200101260004   2001    Jan    26        NaN         0        NaN   
71798  200101280008   2001    Jan    28        NaN         0        NaN   
71837  200102050005   2001    Feb     5        NaN         0        NaN   
71845  200102060005   2001    Feb     6        NaN         0        NaN   
71853  200102070008   2001    Feb     6        NaN         0        NaN   
71879  200102130001   2001    Feb    13        NaN         0        NaN   
71898  200102160004   2001    Feb    16        NaN         0        NaN   
71904  200102180001   2001    Feb    18        NaN         0        NaN   
71925  200102220007   2001    Feb    22        NaN         0        NaN   
71984  200103070001   2001    Mar     7        NaN         0        NaN   
72000  200103090009   2001    Mar     9        NaN         0        NaN   
72045  200103170003   2001    Mar    

In [12]:
'''soviet_union = test[test['country_txt'] == 'Soviet Union']
soviet_union.describe()'''

"soviet_union = test[test['country_txt'] == 'Soviet Union']\nsoviet_union.describe()"

In [25]:
'''random_button = data[
    (data[data['iyear'].notnull()]) & (data[data['imonth'].notnull()]) & (data[data['iday'].notnull()]) & (data[data['country_txt'].notnull()]) &
    (data[data['city'].notnull()]) & (data[data['summary'].notnull()]) & (data[data['attacktype1_txt'].notnull()]) & 
    (data[data['targtype1_txt'].notnull()]) & (data[data['gname'].notnull()]) & (data[data['nkill'].notnull()])], inplace=True'''

"random_button = data[\n    (data[data['iyear'].notnull()]) & (data[data['imonth'].notnull()]) & (data[data['iday'].notnull()]) & (data[data['country_txt'].notnull()]) &\n    (data[data['city'].notnull()]) & (data[data['summary'].notnull()]) & (data[data['attacktype1_txt'].notnull()]) & \n    (data[data['targtype1_txt'].notnull()]) & (data[data['gname'].notnull()]) & (data[data['nkill'].notnull()])], inplace=True"

In [49]:
'''random_button = data[
    (data.iyear.notnull()) & (data.imonth.notnull()) & (data.iday.notnull()) & (data.imonth.notnull()) &
    (data.country_txt.notnull()) & (data.city.notnull()) & (data.summary.notnull()) & (data.attacktype1_txt.notnull()) &
    (data.targsubtype1_txt.notnull()) & (data.gname.notnull()) & (data.nkill.notnull())]'''

'random_button = data[\n    (data.iyear.notnull()) & (data.imonth.notnull()) & (data.iday.notnull()) & (data.imonth.notnull()) &\n    (data.country_txt.notnull()) & (data.city.notnull()) & (data.summary.notnull()) & (data.attacktype1_txt.notnull()) &\n    (data.targsubtype1_txt.notnull()) & (data.gname.notnull()) & (data.nkill.notnull())]'